In [67]:
from utils import *

In [68]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="c643d8c8-4abb-47ba-808c-bf73f60161f1")

# Configure workspace
ws = Workspace.from_config(auth=interactive_auth)

In [69]:
from azureml.core.model import InferenceConfig

env = get_env(name="sentiment-analysis", packages=["inference-schema", "tensorflow", "transformers"])

inference_config = InferenceConfig(
    environment=env,
    source_directory="code",
    entry_script="sentiment_analysis_batch.py",
)

In [70]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [71]:
from azureml.core import Model

service = Model.deploy(
    ws,
    "sentiment-analysis-batch",
    [],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
2022/01/02 17:45:48 Downloading source code...
2022/01/02 17:45:49 Finished downloading source code
2022/01/02 17:45:49 Creating Docker network: acb_default_network, driver: 'bridge'
2022/01/02 17:45:50 Successfully set up Docker network: acb_default_network
2022/01/02 17:45:50 Setting up Docker configuration...
2022/01/02 17:45:50 Successfully set up Docker configuration
2022/01/02 17:45:50 Logging in to registry: b8a9e438956b449d976a3f4cae8455fb.azurecr.io
2022/01/02 17:45:51 Successfully logged into b8a9e438956b449d976a3f4cae8455fb.azurecr.io
2022/01/02 17:45:51 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2022/01/02 17:45:51 Scanning for dependencies...
2022/01/02 17:45:51 Successfully scanned dependencies
2022/01/02 17:45:51 Launching container with name: acb_step_0

Step 1/19 : FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210806.v1@sha256:bb94557d668c1cebffb04d0a77aa461776f69

In [72]:
print(service.get_logs())

2022-01-02T17:54:53,927288700+00:00 - iot-server/run 
2022-01-02T17:54:53,927084900+00:00 - rsyslog/run 
2022-01-02T17:54:53,927489900+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-01-02T17:54:53,932127000+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-02T17:54:54,269492900+00:00 - iot-server/finish 1 0
2022-01-02T17:54:54,271096400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (15)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 43
2022-01-02 17:54:55.815226: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_0b48319fab02ae60c0ce02977a7216a1/lib:/azureml-envs/azureml_0b48319fab02ae60c0ce02977a721

In [73]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    "Inputs": {
        "data": [
            "My ML skills are quite good.",
            "I didn't have good experience with ML."
        ]
    }
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.text)

[0.9998338222503662, -0.9997455477714539]


In [74]:
service.delete()

Container has been successfully cleaned up.
